## Machine Learning With Flowers ##

In [39]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adamax

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [41]:
data = []
for dir, x, files in os.walk('flower_photos'):
    label = dir.split('/')[-1]

    for file in files:
        path = os.path.join(dir, file)
        data.append([path, label])
df = pd.DataFrame(data, columns=['file_path', 'label'])

print(df)

                                             file_path   label
0       flower_photos/roses/16209331331_343c899d38.jpg   roses
1        flower_photos/roses/5777669976_a205f61e5b.jpg   roses
2      flower_photos/roses/4860145119_b1c3cbaa4e_n.jpg   roses
3       flower_photos/roses/15011625580_7974c44bce.jpg   roses
4     flower_photos/roses/17953368844_be3d18cf30_m.jpg   roses
...                                                ...     ...
3665     flower_photos/tulips/134143359_71fa8dd9a4.jpg  tulips
3666    flower_photos/tulips/3637371174_a8dfcc1b35.jpg  tulips
3667  flower_photos/tulips/6948239566_0ac0a124ee_n.jpg  tulips
3668    flower_photos/tulips/2834890466_1cf220fba1.jpg  tulips
3669   flower_photos/tulips/13953090784_0c7d7a904e.jpg  tulips

[3670 rows x 2 columns]


In [42]:
datagen = ImageDataGenerator(
    validation_split = 0.2
)

In [43]:
train_gen = datagen.flow_from_dataframe(
    dataframe = df, 
    x_col = 'file_path',
    y_col = 'label',
    target_size = (224, 224),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True,
    subset = 'training'
)

valid_gen = datagen.flow_from_dataframe(
    dataframe = df, 
    x_col = 'file_path',
    y_col = 'label',
    target_size = (224, 224),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True,
    subset = 'validation'
)

Found 2936 validated image filenames belonging to 5 classes.
Found 734 validated image filenames belonging to 5 classes.


In [44]:
base_model = EfficientNetB3(
    include_top= False, 
    weights='imagenet',
    input_shape=(224, 224, 3), 
    pooling='max'
)

model = tf.keras.models.Sequential([
    base_model,
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(rate=.45, seed=123),
    layers.Dense(5, activation='softmax')
])


model.compile(
    Adamax(learning_rate= .0001), 
    loss = 'categorical_crossentropy',
    metrics=[
        'acc', 
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall(), 
        tf.keras.metrics.AUC()
    ]
)


In [45]:
early_stopping = EarlyStopping(
    patience=10,
    min_delta=0,
    monitor='val_loss',
    verbose=0,
    restore_best_weights=True,
    baseline=None
)

In [38]:
model.fit(
    train_gen,
    validation_data = valid_gen,
    epochs = 5,
    validation_steps=None,
    shuffle=False,
    callbacks=early_stopping,
    verbose=1
)

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 306s 3s/step - acc: 0.8257 - auc: 0.9696 - loss: 0.4879 - precision: 0.8724 - recall: 0.7827 - val_acc: 0.1185 - val_auc: 0.4089 - val_loss: 3.8557 - val_precision: 0.1250 - val_recall: 0.1104
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 282s 3s/step - acc: 0.8661 - auc: 0.9796 - loss: 0.3939 - precision: 0.8993 - recall: 0.8287 - val_acc: 0.1158 - val_auc: 0.3926 - val_loss: 4.3699 - val_precision: 0.1225 - val_recall: 0.1104
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 279s 3s/step - acc: 0.8963 - auc: 0.9866 - loss: 0.3124 - precision: 0.9259 - recall: 0.8719 - val_acc: 0.1144 - val_auc: 0.3829 - val_loss: 4.8066 - val_precision: 0.1202 - val_recall: 0.1104
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 286s 3s/step - acc: 0.9114 - auc: 0.9898 - loss: 0.2667 - precision: 0.9303 - recall: 0.8924 - val_acc: 0.1172 - val_auc: 0.3859 - val_loss: 5.1298 - val_precision: 0.1183 - val_recall: 0.1117
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 277s 3s/step - acc: 0.9296 - auc: 0.9947 - 

In [46]:
model.save('flowers.keras')

In [51]:
import cv2
from tensorflow.keras.saving import load_model
import numpy as np

model = load_model('flowers.keras')

input_image = cv2.imread('flower_photos/tulips/10163955604_ae0b830975_n.jpg')


input_image_resize = cv2.resize(input_image, (224,224))

input_image_scaled = input_image_resize/225
image_reshpaed = np.reshape(input_image_scaled, [1,224,224,3])

input_prediction = model.predict(image_reshpaed)
print(np.argmax(input_prediction))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
4
